# Cricketer class improvements
A development area where I work out new features to build into the various classes to allow for further analysis. 

In [1]:
# Required imports
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

# Import my classes
from classes.Cricketer import Cricketer
from classes.Innings import Innings
from classes.Match import Match

In [2]:
joe_root = 'http://stats.espncricinfo.com/ci/engine/player/303669.html?class=1;template=results;type=allround;view=innings'
root = Cricketer(joe_root)

In [15]:
root_id = '303669'
first_25 = root.innings(True).head(25)

In [4]:
first_innings = first_25.iloc[0]
first_innings_obj = Match(first_innings['match_url']).select_innings(int(first_innings['inns']))

In [10]:
first_innings_scorecard = first_innings_obj.batting()

In [17]:
first_innings_scorecard[first_innings_scorecard.player_id == root_id]

,batsman,how_out,runs,balls_faced,minutes,fours,sixes,strike_rate,is_out,is_keeper,is_captain,player_link,player_id
6,JE Root,c & b Chawla,73,229,289,4,0,31.87,True,False,False,http://www.espncricinfo.com/ci/content/player/...,303669


In [40]:
first_sc_details = first_innings_scorecard[first_innings_scorecard.player_id == root_id]
first_innings['balls_faced'] = int(first_sc_details['balls_faced'].item())
first_innings['position_batted'] = first_sc_details.index.item()
first_innings['minutes'] = first_sc_details.minutes.item()
first_innings['fours'] = first_sc_details.fours.item()
first_innings['sixes'] = first_sc_details.sixes.item()
first_innings['is_captain'] = first_sc_details.is_captain.item()

In [41]:
first_innings

inns                                                               1
score                                                             73
did_bat                                                         True
is_out                                                          True
overs                                                            NaN
conc                                                             NaN
wkts                                                             NaN
did_bowl                                                       False
ct                                                               NaN
st                                                               NaN
opposition                                                     India
ground                                                        Nagpur
start_date                                               13 Dec 2012
match_id                                                      565809
match_url          https://www.esp

With this sort of extra information, we can track other things over time but we will have to start being more aware of the requests that we are doing, as these things take time and resources. 